In [2]:
from itertools import product as cartesianproduct
import time
import pandas as pd

from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from threading import Timer


class TestApp(EWrapper, EClient):
    def __init__(self, date, ticker, lock):
        EClient.__init__(self, self)
        self.date   = date
        self.ticker = ticker
        self.current_index = 0
        self.trade_time, self.trd_px, self.trd_sz, self.exch, self.cond = ([] for _ in range(5))
        self.quote_time, self.bid, self.ask, self.bid_sz, self.ask_sz = ([] for _ in range(5))
        self.last_quote_time = "04:00:00"
        self.last_trade_time = "04:00:00"
        self.reqid = 19000
        self.quotes_done = False
        self.trades_done = False
        
        self.contract = Contract()
        self.contract.symbol = self.ticker.replace('p',' PR')
        self.contract.secType = "STK"
        self.contract.exchange = "SMART"
        self.contract.currency = "USD"
        self.lock = lock
        

    def error(self, reqId, errorCode, errorString):
        if errorCode == 200:
            with self.lock:
                self.quotes_done, self.trades_done = True, True
        print("Error: ", reqId, " ", errorCode, " ", errorString)

    def nextValidId(self, orderId):
        self.start()

    def historicalData(self, reqId, bar):
        print("HistoricalData. ", reqId, " Date:")
        return bar.high


    def start(self):     
        self.reqHistoricalTicks(self.reqid, self.contract, f"{self.date} {self.last_quote_time}", "", 1000, "Bid_ASK", 0, False, [])
        self.reqid += 1
        self.reqHistoricalTicks(self.reqid, self.contract, f"{self.date} {self.last_trade_time}", "", 1000, "TRADES", 0, False, [])
        self.reqid += 1
                                                        
    def stop(self):
        self.done = True
        self.disconnect()
        

    def historicalTicks(self, reqId: int, ticks, done: bool):
        print("mid")
        for tick in ticks:
            print("HistoricalTick. ReqId:", reqId, tick)
    
    def quotes(self):
        return pd.DataFrame({
            'bid': self.bid,
            'ask': self.ask,
            'bid_sz': self.bid_sz,
            'ask_sz': self.ask_sz,
            
        },index=self.quote_time)
    
    def trades(self):
        return pd.DataFrame({            
            'trd_px':self.trd_px, 
            'trd_sz':self.trd_sz, 
            'exch'  :self.exch, 
            'cond'  :self.cond 
            
        },index=self.trade_time)
    
    def get_df(self):
        df = pd.concat([self.quotes(),self.trades()],axis = 0)
        df = df.sort_index()
        df['bid'] = df['bid'].ffill()
        df['bid_sz'] = df['bid_sz'].ffill()
        df['ask'] = df['ask'].ffill()
        df['ask_sz'] = df['ask_sz'].ffill()
        df['exch'] = df['exch'].fillna('')
        df['cond'] = df['cond'].fillna('')    
        return df


    # ! [historicalticksbidask]
    def historicalTicksBidAsk(self, reqId: int, ticks,
                              done: bool):
        if not done:
            print(f"quotes not finished for {self.date} {self.ticker}")
                                    
        if ticks:
            ts = pd.Timestamp(ticks[0].time*1e9,tz = 'US/Eastern')   
            if ts.date() != pd.Timestamp(self.date).date():
                print(f'{self.date} is not a trading day')
                self.quotes_done = True
                return
            
        print(f"{len(ticks)} quote updates for {self.date} {self.ticker}")
                            
        for t in ticks:            
            ts = pd.Timestamp(t.time*1e9,tz = 'US/Eastern')   
            
            self.quote_time.append(ts)
            self.bid.append(t.priceBid)
            self.ask.append(t.priceAsk)
            self.bid_sz.append(t.sizeBid)
            self.ask_sz.append(t.sizeAsk)            
                        
        if len(ticks) >= 1000 :
            self.last_quote_time = str(self.quote_time[-1].time())
            time.sleep(6)            
            self.reqHistoricalTicks(self.reqid, self.contract, f"{self.date} {self.last_quote_time}", "", 1000, "Bid_ASK", 0, False, [])
            self.reqid += 1
        else:
            with self.lock:
                self.quotes_done = True
    

    # ! [historicaltickslast]
    def historicalTicksLast(self, reqId: int, ticks,
                            done: bool):
        if not done:
            print(f"trades finished for {self.date} {self.ticker}")
                                                
        if ticks:
            ts = pd.Timestamp(ticks[0].time*1e9,tz = 'US/Eastern')   
            if ts.date() != pd.Timestamp(self.date).date():
                print(f'{self.date} is not a trading day')
                self.trades_done = True
                return
            
        print(f"{len(ticks)} trades updates for {self.date} {self.ticker}")
    
        for t in ticks:
            ts = pd.Timestamp(t.time*1e9,tz = 'US/Eastern')            
            self.trade_time.append(ts)
            self.trd_px.append(t.price)
            self.trd_sz.append(t.size)
            self.exch.append(t.exchange)
            self.cond.append(t.specialConditions)                      
            
        if len(ticks) >= 1000 :
            self.last_trade_time = str(self.trade_time[-1].time())  
            time.sleep(6)
            print(f"{self.last_trade_time}")
            self.reqHistoricalTicks(self.reqid, self.contract, f"{self.date} {self.last_trade_time}", "", 1000, "TRADES", 0, False, [])
            self.reqid += 1
        else:
            with self.lock:
                self.trades_done = True

    def currentTime(time):
        print(f"current time is {time}")

In [4]:
#tickers = ['PFF']
#tickers = ['NLY PRG'] #['NLY PRG', 'NLY PRF']
#tickers = ['AGNCM', 'AGNCN', 'AGNCO', 'AGNCP']
tickers = ['NLYpG', 'NLYpF', 'NLYpI']
dates = ["".join(str(d.date()).split("-")) for d in pd.date_range('20200630', '20210630')]
dfs = {ticker : [] for ticker in tickers}

In [5]:
import os, threading
lock = threading.Lock()

for dt in dates:
    path = f'../data_test/{dt[0:4]}/{dt[4:6]}/{dt[6:8]}'
    try:
        os.makedirs(path)
    except Exception as e:
        print(e)
    for ticker in tickers:    
        print(f'{dt} {ticker}')
        app = TestApp(dt, ticker, lock)
        app.nextOrderId = 0
        app.connect("127.0.0.1", 7496, 0)
        Timer(1, app.run).start()

        while True:
            with lock :
                if app.trades_done and app.quotes_done:
                    break
            time.sleep(1.0)            
        app.stop()
        df = app.get_df()
        if df.shape[0] > 0:                 
            df.to_hdf(f'{path}/{ticker}.h5',key='data')

[WinError 183] Cannot create a file when that file already exists: '../data_test/2020/06/30'
20200630 NLYpG
Error:  -1   2104   Market data farm connection is OK:usfarm.nj
Error:  -1   2104   Market data farm connection is OK:hfarm
Error:  -1   2104   Market data farm connection is OK:cashfarm
Error:  -1   2104   Market data farm connection is OK:usopt
Error:  -1   2104   Market data farm connection is OK:usfarm
Error:  -1   2106   HMDS data farm connection is OK:hkhmds
Error:  -1   2106   HMDS data farm connection is OK:fundfarm
Error:  -1   2106   HMDS data farm connection is OK:ushmds
Error:  -1   2158   Sec-def data farm connection is OK:secdefhk
760 quote updates for 20200630 NLYpG
288 trades updates for 20200630 NLYpG
20200630 NLYpF
Error:  -1   2104   Market data farm connection is OK:usfarm.nj
Error:  -1   2104   Market data farm connection is OK:hfarm
Error:  -1   2104   Market data farm connection is OK:cashfarm
Error:  -1   2104   Market data farm connection is OK:usopt
Erro

20200704 is not a trading day
20200704 is not a trading day
20200704 NLYpF
Error:  -1   2104   Market data farm connection is OK:usfarm.nj
Error:  -1   2104   Market data farm connection is OK:hfarm
Error:  -1   2104   Market data farm connection is OK:cashfarm
Error:  -1   2104   Market data farm connection is OK:usopt
Error:  -1   2104   Market data farm connection is OK:usfarm
Error:  -1   2106   HMDS data farm connection is OK:hkhmds
Error:  -1   2106   HMDS data farm connection is OK:fundfarm
Error:  -1   2106   HMDS data farm connection is OK:ushmds
Error:  -1   2158   Sec-def data farm connection is OK:secdefhk
20200704 is not a trading day
20200704 is not a trading day
20200704 NLYpI
Error:  -1   2104   Market data farm connection is OK:usfarm.nj
Error:  -1   2104   Market data farm connection is OK:hfarm
Error:  -1   2104   Market data farm connection is OK:cashfarm
Error:  -1   2104   Market data farm connection is OK:usopt
Error:  -1   2104   Market data farm connection is O

466 quote updates for 20200708 NLYpF
315 trades updates for 20200708 NLYpF
20200708 NLYpI
Error:  -1   2104   Market data farm connection is OK:usfarm.nj
Error:  -1   2104   Market data farm connection is OK:hfarm
Error:  -1   2104   Market data farm connection is OK:cashfarm
Error:  -1   2104   Market data farm connection is OK:usopt
Error:  -1   2104   Market data farm connection is OK:usfarm
Error:  -1   2106   HMDS data farm connection is OK:hkhmds
Error:  -1   2106   HMDS data farm connection is OK:fundfarm
Error:  -1   2106   HMDS data farm connection is OK:ushmds
Error:  -1   2158   Sec-def data farm connection is OK:secdefhk
992 quote updates for 20200708 NLYpI
144 trades updates for 20200708 NLYpI
[WinError 183] Cannot create a file when that file already exists: '../data_test/2020/07/09'
20200709 NLYpG
Error:  -1   2104   Market data farm connection is OK:usfarm.nj
Error:  -1   2104   Market data farm connection is OK:hfarm
Error:  -1   2104   Market data farm connection is O

20200712 is not a trading day
20200712 is not a trading day
[WinError 183] Cannot create a file when that file already exists: '../data_test/2020/07/13'
20200713 NLYpG
Error:  -1   2104   Market data farm connection is OK:usfarm.nj
Error:  -1   2104   Market data farm connection is OK:hfarm
Error:  -1   2104   Market data farm connection is OK:cashfarm
Error:  -1   2104   Market data farm connection is OK:usopt
Error:  -1   2104   Market data farm connection is OK:usfarm
Error:  -1   2106   HMDS data farm connection is OK:hkhmds
Error:  -1   2106   HMDS data farm connection is OK:fundfarm
Error:  -1   2106   HMDS data farm connection is OK:ushmds
Error:  -1   2158   Sec-def data farm connection is OK:secdefhk
270 quote updates for 20200713 NLYpG
229 trades updates for 20200713 NLYpG
20200713 NLYpF
Error:  -1   2104   Market data farm connection is OK:usfarm.nj
Error:  -1   2104   Market data farm connection is OK:hfarm
Error:  -1   2104   Market data farm connection is OK:cashfarm
Erro

Error:  -1   2104   Market data farm connection is OK:usfarm.nj
Error:  -1   2104   Market data farm connection is OK:hfarm
Error:  -1   2104   Market data farm connection is OK:cashfarm
Error:  -1   2104   Market data farm connection is OK:usopt
Error:  -1   2104   Market data farm connection is OK:usfarm
Error:  -1   2106   HMDS data farm connection is OK:hkhmds
Error:  -1   2106   HMDS data farm connection is OK:fundfarm
Error:  -1   2106   HMDS data farm connection is OK:ushmds
Error:  -1   2158   Sec-def data farm connection is OK:secdefhk
333 quote updates for 20200717 NLYpG
284 trades updates for 20200717 NLYpG
20200717 NLYpF
Error:  -1   2104   Market data farm connection is OK:usfarm.nj
Error:  -1   2104   Market data farm connection is OK:hfarm
Error:  -1   2104   Market data farm connection is OK:cashfarm
Error:  -1   2104   Market data farm connection is OK:usopt
Error:  -1   2104   Market data farm connection is OK:usfarm
Error:  -1   2106   HMDS data farm connection is OK

...

20210613 is not a trading day
20210613 is not a trading day
[WinError 183] Cannot create a file when that file already exists: '../data_test/2021/06/14'
20210614 NLYpG
Error:  -1   2104   Market data farm connection is OK:usfarm.nj
Error:  -1   2104   Market data farm connection is OK:hfarm
Error:  -1   2104   Market data farm connection is OK:cashfarm
Error:  -1   2104   Market data farm connection is OK:usopt
Error:  -1   2104   Market data farm connection is OK:usfarm
Error:  -1   2106   HMDS data farm connection is OK:hkhmds
Error:  -1   2106   HMDS data farm connection is OK:fundfarm
Error:  -1   2106   HMDS data farm connection is OK:ushmds
Error:  -1   2158   Sec-def data farm connection is OK:secdefhk
568 quote updates for 20210614 NLYpG
145 trades updates for 20210614 NLYpG
20210614 NLYpF
Error:  -1   2104   Market data farm connection is OK:usfarm.nj
Error:  -1   2104   Market data farm connection is OK:hfarm
Error:  -1   2104   Market data farm connection is OK:cashfarm
Erro

Error:  -1   2104   Market data farm connection is OK:usfarm.nj
Error:  -1   2104   Market data farm connection is OK:hfarm
Error:  -1   2104   Market data farm connection is OK:cashfarm
Error:  -1   2104   Market data farm connection is OK:usopt
Error:  -1   2104   Market data farm connection is OK:usfarm
Error:  -1   2106   HMDS data farm connection is OK:hkhmds
Error:  -1   2106   HMDS data farm connection is OK:fundfarm
Error:  -1   2106   HMDS data farm connection is OK:ushmds
Error:  -1   2158   Sec-def data farm connection is OK:secdefhk
550 quote updates for 20210618 NLYpG
183 trades updates for 20210618 NLYpG
20210618 NLYpF
Error:  -1   2104   Market data farm connection is OK:usfarm.nj
Error:  -1   2104   Market data farm connection is OK:hfarm
Error:  -1   2104   Market data farm connection is OK:cashfarm
Error:  -1   2104   Market data farm connection is OK:usopt
Error:  -1   2104   Market data farm connection is OK:usfarm
Error:  -1   2106   HMDS data farm connection is OK

540 quote updates for 20210622 NLYpG
96 trades updates for 20210622 NLYpG
20210622 NLYpF
Error:  -1   2104   Market data farm connection is OK:usfarm.nj
Error:  -1   2104   Market data farm connection is OK:hfarm
Error:  -1   2104   Market data farm connection is OK:cashfarm
Error:  -1   2104   Market data farm connection is OK:usopt
Error:  -1   2104   Market data farm connection is OK:usfarm
Error:  -1   2106   HMDS data farm connection is OK:hkhmds
Error:  -1   2106   HMDS data farm connection is OK:fundfarm
Error:  -1   2106   HMDS data farm connection is OK:ushmds
Error:  -1   2158   Sec-def data farm connection is OK:secdefhk
731 quote updates for 20210622 NLYpF
387 trades updates for 20210622 NLYpF
20210622 NLYpI
Error:  -1   2104   Market data farm connection is OK:usfarm.nj
Error:  -1   2104   Market data farm connection is OK:hfarm
Error:  -1   2104   Market data farm connection is OK:cashfarm
Error:  -1   2104   Market data farm connection is OK:usopt
Error:  -1   2104   Mar

20210626 is not a trading day
20210626 is not a trading day
20210626 NLYpI
Error:  -1   2104   Market data farm connection is OK:usfarm.nj
Error:  -1   2104   Market data farm connection is OK:hfarm
Error:  -1   2104   Market data farm connection is OK:cashfarm
Error:  -1   2104   Market data farm connection is OK:usopt
Error:  -1   2104   Market data farm connection is OK:usfarm
Error:  -1   2106   HMDS data farm connection is OK:hkhmds
Error:  -1   2106   HMDS data farm connection is OK:fundfarm
Error:  -1   2106   HMDS data farm connection is OK:ushmds
Error:  -1   2158   Sec-def data farm connection is OK:secdefhk
20210626 is not a trading day
20210626 is not a trading day
[WinError 183] Cannot create a file when that file already exists: '../data_test/2021/06/27'
20210627 NLYpG
Error:  -1   2104   Market data farm connection is OK:usfarm.nj
Error:  -1   2104   Market data farm connection is OK:hfarm
Error:  -1   2104   Market data farm connection is OK:cashfarm
Error:  -1   2104  

1003 quote updates for 20210630 NLYpI
475 trades updates for 20210630 NLYpI
212 quote updates for 20210630 NLYpI
